## 네이버 카페 완성 코드 ver.2
- 키워드 없이 전체글 기준으로 크롤링하는 버전임


- 현재 코드에서 크롤링 대상: 편측성 난청 아이들의 엄마 

+ 진행상황 추가 코드 

In [ ]:
import time
import os
import traceback
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import pandas as pd
from datetime import datetime

# ===== 1. 초기 설정 =====
YOUR_ID = ""      # 👉 실제 아이디로 변경
YOUR_PW = ""    # 👉 실제 비밀번호로 변경

CAFE_URL = "https://cafe.naver.com/itsokssd"
CLUB_ID = "28211985"  # itsokssd 카페 clubid

out_dir = "itsokssd_cafe_result"
os.makedirs(out_dir, exist_ok=True)

error_log_file = os.path.join(out_dir, "itsokssd_crawl_errors.log")
open(error_log_file, "w", encoding="utf-8").close()  # 실행 시 기존 로그 초기화


def log_error(stage, msg="", url=None, exc: Exception | None = None):
    """에러 로그 파일에 기록하는 함수"""
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lines = [f"[{ts}] [{stage}]"]
    if url:
        lines.append(f"url={url}")
    if msg:
        lines.append(f"msg={msg}")
    if exc:
        lines.append("traceback:")
        lines.append(traceback.format_exc())
    log = " | ".join(lines) + "\n"

    with open(error_log_file, "a", encoding="utf-8") as f:
        f.write(log)


# ===== 2. webdriver 실행 및 로그인 =====
print("웹드라이버를 실행하고 네이버에 로그인")
driver = wb.Chrome()
driver.maximize_window()

driver.get("https://nid.naver.com/nidlogin.login")
driver.implicitly_wait(10)

try:
    driver.execute_script(
        f"document.getElementsByName('id')[0].value = '{YOUR_ID}'"
    )
    time.sleep(0.5)
    driver.execute_script(
        f"document.getElementsByName('pw')[0].value = '{YOUR_PW}'"
    )
    time.sleep(0.5)
    driver.find_element(By.ID, "log.login").click()
    time.sleep(3)
    print("✅ 로그인 완료!")
except Exception as e:
    log_error(
        stage="로그인",
        msg="로그인 중 오류 발생",
        exc=e,
        url=driver.current_url,
    )
    driver.quit()
    raise SystemExit


# ===== 3. 카페 접속 & 전체글 보기 진입 =====
try:
    driver.get(CAFE_URL)
    driver.implicitly_wait(10)
    time.sleep(2)

    # '전체글보기' 메뉴 클릭
    driver.find_element(By.ID, "menuLink0").click()
    driver.implicitly_wait(10)
    time.sleep(2)

    # 페이지당 게시글 개수 50개로 변경
    try:
        driver.find_element(
            By.XPATH,
            '//*[@id="cafe_content"]/div[2]/div/div[3]/div/button'
        ).click()
        time.sleep(0.5)
        driver.find_element(
            By.XPATH,
            '//*[@id="cafe_content"]/div[2]/div/div[3]/div[2]/ul/li[7]'
        ).click()
        print("📌 게시물 표시 개수를 '50개'로 설정 완료")
        time.sleep(1)
    except NoSuchElementException:
        log_error(stage="게시글 설정", msg="게시글 보기 버튼 없음", url=driver.current_url)

except Exception as e:
    log_error(stage="카페 진입", msg="카페 진입 실패", exc=e, url=driver.current_url)
    driver.quit()
    raise SystemExit


# ===== 4. 페이지네이션 기반으로 전체 게시글 URL 수집 =====
page_url_list = []

try:
    # 1) 전체글 1페이지 URL (템플릿 확인용)
    first_page_url = (
        f"https://cafe.naver.com/f-e/cafes/{CLUB_ID}/menus/0"
        f"?viewType=L&page=1&size=50"
    )
    driver.get(first_page_url)
    driver.implicitly_wait(10)
    time.sleep(1)

    # 2) 마지막 페이지 수동 지정 (네가 직접 카페에서 확인한 값)
    last_page = 156  # 👈 itsokssd 기준으로 네가 확인한 값
    print(f"📄 전체글 마지막 페이지(수동 지정): {last_page}")

    # 3) 각 페이지 돌면서 .article 에서 href 수집
    empty_streak = 0

    for i in tqdm(range(1, last_page + 1), desc="[전체글] 페이지 이동 중", ncols=80):
        url = (
            f"https://cafe.naver.com/f-e/cafes/{CLUB_ID}/menus/0"
            f"?viewType=L&page={i}&size=50"
        )
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(2)

        try:
            if driver.find_elements(By.ID, "cafe_main"):
                driver.switch_to.frame("cafe_main")
                time.sleep(0.5)

            aTags = driver.find_elements(By.CSS_SELECTOR, "a.article")
            new_cnt = 0
            for tag in aTags:
                href = tag.get_attribute("href")
                if href:
                    page_url_list.append(href)
                    new_cnt += 1

            driver.switch_to.default_content()

            if new_cnt == 0:
                empty_streak += 1
                log_error(
                    stage="URL 수집",
                    msg=f"빈 페이지 감지 p={i}",
                    url=url,
                )
            else:
                empty_streak = 0

        except Exception as e:
            driver.switch_to.default_content()
            log_error(
                stage="URL 수집",
                msg="예상치 못한 오류",
                exc=e,
                url=url,
            )

except Exception as e:
    log_error(
        stage="URL 수집",
        msg="전체 URL 수집 중 예외",
        exc=e,
        url=driver.current_url,
    )

# 4) 중복 제거
page_url_list = list(dict.fromkeys(page_url_list))
print(f"\n🔗 총 {len(page_url_list)}개의 게시글 URL 수집 완료.")
print("🔎 URL 예시 5개:")
for u in page_url_list[:5]:
    print("  -", u)


# ===== 5. 본문 크롤링 설정 =====
SKIP_BOARDS = [
    "가입인사",
    "공지사항",
]

df = []

# 테스트용으로 일부만 돌리고 싶으면 아래 주석 해제
# page_url_list = page_url_list[:50]

for idx, href in enumerate(
    tqdm(page_url_list, desc="[본문 크롤링 중]", ncols=80),
    start=1
):
    try:
        driver.get(href)
        driver.implicitly_wait(10)
        time.sleep(1.5)

        # iframe 있으면 진입
        if driver.find_elements(By.ID, "cafe_main"):
            driver.switch_to.frame("cafe_main")
            time.sleep(0.5)

        # 1) 게시판명
        try:
            try:
                board = driver.find_element(
                    By.CSS_SELECTOR, "a.link_board"
                ).text.strip()
            except NoSuchElementException:
                board = driver.find_element(
                    By.CSS_SELECTOR,
                    "span.category, span.brd_name, a#linkBoard"
                ).text.strip()
        except:
            board = None

        # 가입인사/공지사항 등 스킵
        if board in SKIP_BOARDS:
            driver.switch_to.default_content()
            continue

        # 2) 제목
        try:
            title = driver.find_element(
                By.CSS_SELECTOR, ".title_text"
            ).text.strip()
        except NoSuchElementException:
            title = None

        # 3) 날짜
        try:
            date = driver.find_element(
                By.CLASS_NAME, "date"
            ).text.strip()
        except NoSuchElementException:
            date = None

        # 4) 내용
        content = None
        try:
            content = driver.find_element(
                By.CSS_SELECTOR, ".se-main-container"
            ).text.strip()
        except:
            pass

        if not content:
            try:
                content = driver.find_element(
                    By.CSS_SELECTOR, "#tbody"
                ).text.strip()
            except:
                pass

        if not content:
            try:
                content = driver.find_element(
                    By.TAG_NAME, "body"
                ).text.strip()
            except:
                content = None

 
        df.append([board, title, content, date, href])

        # 100개마다 진행 상황 출력
        if idx % 100 == 0:
            print(
                f"   🔹 현재까지 본문 크롤링 완료: {idx} / {len(page_url_list)}"
            )

    except Exception as e:
        driver.switch_to.default_content()
        log_error(
            stage="본문 크롤링",
            msg="알 수 없는 오류",
            url=href,
            exc=e,
        )
        # 예외 시에도 row 구조 유지
        df.append([None, None, None, None, href])

    finally:
        # iframe 빠져나오기
        try:
            driver.switch_to.default_content()
        except:
            pass


print("\n📥 본문 크롤링 완료, df row 수:", len(df))


# ===== 6. DataFrame 생성 + 저장 =====
cafe_df = pd.DataFrame(
    df,
    columns=["board", "title", "content", "date", "href"]
)
print("📊 원본 df:", cafe_df.shape)

# 카테고리별 글 개수 확인
print("\n📊 게시판(카테고리)별 글 개수 TOP 10")
print(cafe_df["board"].value_counts().head(10))

# 제목 + 내용이 있는 경우만 사용
cafe_df_cleaned = cafe_df.dropna(subset=["title", "content"], how="any")
cafe_df_cleaned = cafe_df_cleaned.drop_duplicates(subset=["href"])

print("🧹 클린 df:", cafe_df_cleaned.shape)

# 저장
csv_path = os.path.join(out_dir, "itsokssd_all_posts.csv")
pkl_path = os.path.join(out_dir, "itsokssd_all_posts.pkl")

cafe_df_cleaned.to_csv(csv_path, index=False, encoding="utf-8-sig")
cafe_df_cleaned.to_pickle(pkl_path)

print("\n✅ CSV 저장:", csv_path)
print("✅ PKL 저장:", pkl_path)

driver.quit()
print("\n🎉 모든 작업 완료!")


웹드라이버를 실행하고 네이버에 로그인
✅ 로그인 완료!
📌 게시물 표시 개수를 '50개'로 설정 완료
📄 전체글 마지막 페이지(수동 지정): 156


[전체글] 페이지 이동 중: 100%|████████████████| 156/156 [33:23<00:00, 12.84s/it]



🔗 총 7762개의 게시글 URL 수집 완료.
🔎 URL 예시 5개:
  - https://cafe.naver.com/f-e/cafes/28211985/articles/8276?boardtype=L&userDisplay=50&referrerAllArticles=false&page=1
  - https://cafe.naver.com/f-e/cafes/28211985/articles/1077?boardtype=L&userDisplay=50&referrerAllArticles=false&page=1
  - https://cafe.naver.com/f-e/cafes/28211985/articles/602?boardtype=L&userDisplay=50&referrerAllArticles=false&page=1
  - https://cafe.naver.com/f-e/cafes/28211985/articles/682?boardtype=L&userDisplay=50&referrerAllArticles=false&page=1
  - https://cafe.naver.com/f-e/cafes/28211985/articles/575?boardtype=L&userDisplay=50&referrerAllArticles=false&page=1


[본문 크롤링 중]:   5%|█                   | 400/7762 [22:45<5:43:32,  2.80s/it]

   🔹 현재까지 본문 크롤링 완료: 400 / 7762


...

   🔹 현재까지 본문 크롤링 완료: 7500 / 7762


[본문 크롤링 중]:  98%|███████████████▋| 7600/7762 [20:24:05<1:06:28, 24.62s/it]

   🔹 현재까지 본문 크롤링 완료: 7600 / 7762


[본문 크롤링 중]: 100%|██████████████████| 7762/7762 [21:40:14<00:00, 10.05s/it]



📥 본문 크롤링 완료, df row 수: 4042
📊 원본 df: (4042, 5)

📊 게시판(카테고리)별 글 개수 TOP 10
board
우리 사는 이야기(일반이상)    1348
질문있어요               819
우리 아이는요~            538
한줄 수다               441
첫돌까지                200
SSD 관련 정보            83
서울경기                 81
경상                   43
초등                   41
세돌까지                 36
Name: count, dtype: int64
🧹 클린 df: (3713, 5)

✅ CSV 저장: itsokssd_cafe_result\itsokssd_all_posts.csv
✅ PKL 저장: itsokssd_cafe_result\itsokssd_all_posts.pkl

🎉 모든 작업 완료!
